In [ ]:
import nltk
from gensim import corpora, models
nltk.download('brown')
from nltk.corpus import brown
from nltk.corpus import wordnet as wn
import networkx as nx
from nltk.metrics import edit_distance
from nltk.metrics import jaccard_distance
nltk.download('omw-1.4')
nltk.download('punkt')
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk import stem
stemmer = stem.PorterStemmer()
from nltk import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
import string
nltk.download('vader_lexicon')
punct = list(string.punctuation)
!pip install PRAW
from mpl_toolkits.mplot3d import Axes3D
import praw
import datetime
import nltk.sentiment.vader as vd
import re
import pandas as pd
import seaborn as sns
sns.set()
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from IPython.display import IFrame
import random
from collections import Counter
import random
from random import sample
from scipy.stats import entropy
import requests
import os
import glob
from tqdm import tqdm
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vader_lexicon = SentimentIntensityAnalyzer()
from wordcloud import WordCloud
!pip install wordcloud
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objs as go
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
import plotly.express as px
!pip install spacy
!python -m spacy download en_core_web_sm
import spacy
from sklearn.cluster import KMeans
!pip install gensim
import gensim
from gensim import corpora
from gensim import corpora, models

In [ ]:
#increasing the bandwidth of the jupiter notebook to allow for the qaunitities of data
%config NotebookApp.iopub_data_rate_limit = 10000000

# FICTION TEXTS

In [ ]:
# Define a dictionary containing the search parameters for the Gutendex API
params_g = {'topic':'fiction'}

# Define the base URL for the Gutendex API
gut = 'https://gutendex.com/books/'

# Send a GET request to the Gutendex API with the search parameters
rg = requests.get(url = gut, params = params_g)

# Get the response from the API in JSON format
rg = rg.json()


In [ ]:
# Define the path to the desktop directory where the text files will be saved
desktop_path = '/filepathname/'

# Define a list of URLs for the text files to be downloaded
urls = [
    'https://www.gutenberg.org/ebooks/84.txt.utf-8',
    'https://www.gutenberg.org/ebooks/36.txt.utf-8',
    'https://www.gutenberg.org/ebooks/42324.txt.utf-8',
    'https://www.gutenberg.org/ebooks/41445.txt.utf-8',
    'https://www.gutenberg.org/ebooks/21279.txt.utf-8',
    'https://www.gutenberg.org/ebooks/164.txt.utf-8',
    'https://www.gutenberg.org/ebooks/5230.txt.utf-8',
    'https://www.gutenberg.org/ebooks/159.txt.utf-8',
    'https://www.gutenberg.org/ebooks/32032.txt.utf-8',
    'https://www.gutenberg.org/ebooks/61309.txt.utf-8',
    'https://www.gutenberg.org/ebooks/86.txt.utf-8',
    'https://www.gutenberg.org/ebooks/69762.txt.utf-8',
    'https://www.gutenberg.org/ebooks/139.txt.utf-8',
    'https://www.gutenberg.org/ebooks/201.txt.utf-8',
    'https://www.gutenberg.org/ebooks/18857.txt.utf-8',
    'https://www.gutenberg.org/ebooks/69703.txt.utf-8',
    'https://www.gutenberg.org/ebooks/19141.txt.utf-8',
    'https://www.gutenberg.org/ebooks/61963.txt.utf-8',
    
]

# Loop through each URL in the list of URLs
for url in urls:
    # Send a GET request to the URL to download the text file
    response = requests.get(url)
    # Get the filename from the URL and use it to create the name of the local file
    filename = f"{url.split('/')[-1].split('.')[0]}.txt"
    # Combine the desktop path with the filename to create the full path to the local file
    filepath = os.path.join(desktop_path, filename)
    # Write the contents of the downloaded text file to the local file
    with open(filepath, "w") as f:
        f.write(response.text)

In [ ]:
# create an empty dataframe to hold the data
df = pd.DataFrame()

# loop through each text file and read it into a pandas dataframe
for file in glob.glob("//filepathname/*.txt"):
    with open(file, "r") as f:
        content = f.read()
        # create a new dataframe with the file contents and add it to the main dataframe
        df_new = pd.DataFrame({'text': [content]})
        df = pd.concat([df, df_new], ignore_index=True)


In [ ]:
# Applying ascii to entire dataframe
df = df.applymap(lambda x: x.encode('ascii', 'ignore').decode('ascii') if isinstance(x, str) else x)

In [ ]:
# Create a new DataFrame with a single row that contains all the text from the original DataFrame
df = pd.DataFrame({
    'text_combined': [df['text'].sum()]  # Sum all the strings in the 'text' column and put the result in a list
})


In [ ]:
# Split the text into individual words
df['words'] = df['text_combined'].str.split()

# Create a new DataFrame with one row for each word
new_df = df.explode('words')

# Reset the index of the new DataFrame
new_df = new_df.reset_index(drop=True)


In [ ]:
# Define function to convert NLTK part-of-speech tags to WordNet part-of-speech tags
def get_wordnet_pos(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # By default, assume the word is a noun

# Create lemmatizer object
lemmatizer = WordNetLemmatizer()

# Tokenize and lemmatize the 'text' column, and store the result in a new column called 'tokenized_words'
new_df['tokenized_words'] = new_df['words'].str.lower().apply(nltk.word_tokenize).apply(nltk.pos_tag).apply(lambda x: [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in x])


# FICTION ANALYSIS VAD, TI-IDF

In [ ]:
# Load the VAD scores from the Warriner rescale
vad = pd.read_csv('//filepathname//Warriner_rescale.csv', index_col=0)
vad = vad[['V.Mean.Sum', 'A.Mean.Sum', 'D.Mean.Sum']]
vad.columns = ['valence', 'arousal', 'dominance']

# Define a function to get the VAD scores for a given word
def get_vad_scores(word):
    if word.lower() in vad.index:
        scores = vad.loc[word.lower()].values
        return scores
    else:
        return [np.nan, np.nan, np.nan]
    
# Create a new column in the dataframe to store the VAD scores
new_df['vad_scores'] = [[] for _ in range(len(new_df))]

# Iterate over each row in the dataframe and calculate VAD scores for each tokenized word
for i, row in new_df.iterrows():
    vad_scores = []
    for word in row['tokenized_words']:
        vad_scores.append(get_vad_scores(word))
    new_df.at[i, 'vad_scores'] = vad_scores


In [ ]:
# Define a function named get_vad_scores that takes a word as input
def get_vad_scores(word):
    # Try to get the VAD scores for the word from the vad DataFrame
    try:
        # Lookup the word in the vad DataFrame and retrieve the VAD scores as a NumPy array
        # The .lower() method is used to convert the word to lowercase for case-insensitive matching
        scores = vad.loc[word.lower()].values
        # Return the VAD scores as a NumPy array
        return scores
    # If the word is not in the vad DataFrame, catch the KeyError exception
    except KeyError:
        # Return None if the word is not in the vad DataFrame
        return None


In [ ]:
#Initialize empty lists to store the VAD scores for each word in the tokenized_words column of new_df

valence_scores = []
arousal_scores = []
dominance_scores = []

#Iterate over each row in new_df

for i, row in new_df.iterrows():
# Iterate over each word in the tokenized_words column of the current row
    for word in row['tokenized_words']:
    # Call the get_vad_scores function to get the VAD scores for the current word
        score = get_vad_scores(word)
        # If the get_vad_scores function returns a valid VAD score, append the scores to the appropriate list
        if score is not None:
            valence_scores.append(score[0])
            arousal_scores.append(score[1])
            dominance_scores.append(score[2])

In [ ]:
# Calculate the mean of the valence_scores list
mean_valence = np.mean(valence_scores)

# Calculate the median of the valence_scores list
median_valence = np.median(valence_scores)

# Calculate the standard deviation of the valence_scores list
std_valence = np.std(valence_scores)

# Print the calculated measures
print("Mean Valence Score:", mean_valence)
print("Median Valence Score:", median_valence)
print("Standard Deviation of Valence Scores:", std_valence)


In [ ]:
# Linear regression on valence scores
X = np.array(valence_scores).reshape(-1, 1)
y = np.array(dominance_scores)  # replace dominance_scores with your target variable
model = LinearRegression().fit(X, y)
r_sq = model.score(X, y)

print("R-squared value:", r_sq)

In [ ]:
# Linear regression on arousal scores
X = np.array(arousal_scores).reshape(-1, 1)
y = np.array(dominance_scores)  # replace dominance_scores with your target variable
model = LinearRegression().fit(X, y)
r_sq = model.score(X, y)

print("R-squared value:", r_sq)

In [ ]:
# Linear regression on arousal scores
X = np.array(valence_scores).reshape(-1, 1)
y = np.array(arousal_scores)  # replace dominance_scores with your target variable
model = LinearRegression().fit(X, y)
r_sq = model.score(X, y)

print("R-squared value:", r_sq)

In [ ]:
# create a scatter plot of valence and arousal scores
plt.scatter(valence_scores, arousal_scores, color='blue', alpha=0.5)

# set the axis labels and title
plt.xlabel('Valence')
plt.ylabel('Arousal')
plt.title('Fiction VAD Scores')

# show the plot
plt.show()


In [ ]:
# create a scatter plot of valence and dominance scores
plt.scatter(valence_scores, dominance_scores, color='blue', alpha=0.5)

# set the axis labels and title
plt.xlabel('Valence')
plt.ylabel('Dominance')
plt.title('Fiction VAD Scores')

# show the plot
plt.show()

In [ ]:
# create a scatter plot of arousal and dominance scores
plt.scatter(arousal_scores, dominance_scores, color='blue', alpha=0.5)

# set the axis labels and title
plt.xlabel('Arousal')
plt.ylabel('Dominance')
plt.title('Fiction VAD Scores')

# show the plot
plt.show()

In [ ]:
# Create a new figure for the 3D plot
fig = plt.figure()

# Create a new subplot with 3D projection
ax = fig.add_subplot(111, projection='3d')

# Plot the VAD scores for each word as a scatter plot
ax.scatter(valence_scores, arousal_scores, dominance_scores)

# Set the labels for the x, y, and z axes
ax.set_xlabel('Valence')
ax.set_ylabel('Arousal')
ax.set_zlabel('Dominance')

# Display the plot
plt.show()


In [ ]:
# Create a trace for the scatter plot
trace = go.Scatter3d(
    x=valence_scores,
    y=arousal_scores,
    z=dominance_scores,
    mode='markers',
    marker=dict(
        size=5,
        color=valence_scores,  # Set color to valence scores
        colorscale='Viridis',  # Set the color scale
        opacity=0.8
    )
)

# Create the layout for the plot
layout = go.Layout(
    scene=dict(
        xaxis=dict(title='Valence'),
        yaxis=dict(title='Arousal'),
        zaxis=dict(title='Dominance')
    ),
    margin=dict(l=0, r=0, b=0, t=0)
)

# Create the figure and add the trace and layout
fig = go.Figure(data=[trace], layout=layout)

# Display the plot
fig.show()


In [ ]:
# get a random sample of 10 documents
documents = new_df['text_combined'].sample(n=10).tolist()

# create a list of stop words
stop_words = set(stopwords.words('english'))

# tokenize each document and remove stop words
texts = [[word for word in word_tokenize(document.lower()) if word not in stop_words] for document in documents]

# create a dictionary from the tokenized texts
dictionary = corpora.Dictionary(texts)

# create a corpus from the dictionary and tokenized texts
corpus = [dictionary.doc2bow(text) for text in texts]

# build the LDA model with 5 topics
num_topics = 5
lda_model = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, passes=10, random_state=42)

# print the top 10 words for each topic
for i, topic in lda_model.show_topics(num_topics=num_topics, num_words=10, formatted=False):
    print(f'Topic {i}: {" ".join([word[0] for word in topic])}')


In [ ]:
# create the matrix
doc_topic_matrix = []
for i, doc in enumerate(corpus):
    topic_probs = lda_model.get_document_topics(doc, minimum_probability=0)
    topic_probs_dict = dict(topic_probs)
    row = [topic_probs_dict.get(j, 0) for j in range(num_topics)]
    doc_topic_matrix.append(row)

# create the heatmap
sns.heatmap(doc_topic_matrix, cmap='YlGnBu')
plt.xlabel('Topic')
plt.ylabel('Document')
plt.title(f'Topic Probabilities for {len(documents)} Documents')
plt.show()


In [ ]:
# define the file path
file_path = "/Users/felixhawkings/Desktop/csv/new_df.csv"

# export the dataframe to CSV
new_df.to_csv(file_path, index=False)

# check if the file was saved successfully
if os.path.exists(file_path):
    print(f"File saved successfully at {file_path}")
else:
    print(f"Error: file not saved at {file_path}")


# SCIENCE FICTION TEXT

In [ ]:
# Define a dictionary containing the search parameters for the Gutendex API
params_g = {'topic':'science fiction'}

# Define the base URL for the Gutendex API
gut = 'https://gutendex.com/books/'

# Send a GET request to the Gutendex API with the search parameters
rg = requests.get(url = gut, params = params_g)

# Get the response from the API in JSON format
rg = rg.json()

In [ ]:
# Define the path to the desktop directory where the text files will be saved
desktop_path = '/filepathname/'

# Define a list of URLs for the text files to be downloaded
urls = ['https://www.gutenberg.org/ebooks/2641.txt.utf-8',
 'https://www.gutenberg.org/ebooks/145.txt.utf-8',
 'https://www.gutenberg.org/ebooks/37106.txt.utf-8',
 'https://www.gutenberg.org/ebooks/16389.txt.utf-8',
 'https://www.gutenberg.org/ebooks/67979.txt.utf-8',
 'https://www.gutenberg.org/ebooks/6761.txt.utf-8',
 'https://www.gutenberg.org/ebooks/394.txt.utf-8',
 'https://www.gutenberg.org/ebooks/1259.txt.utf-8',
 'https://www.gutenberg.org/ebooks/84.txt.utf-8',
 'https://www.gutenberg.org/ebooks/1342.txt.utf-8',
 'https://www.gutenberg.org/ebooks/11.txt.utf-8',
 'https://www.gutenberg.org/ebooks/174.txt.utf-8',
 'https://www.gutenberg.org/ebooks/64317.txt.utf-8',
 'https://www.gutenberg.org/ebooks/46.txt.utf-8',
 'https://www.gutenberg.org/ebooks/345.txt.utf-8',
 'https://www.gutenberg.org/ebooks/25344.txt.utf-8',
 'https://www.gutenberg.org/ebooks/20228.txt.utf-8',
 'https://www.gutenberg.org/ebooks/28054.txt.utf-8'
]

# Loop through each URL in the list of URLs
for url in urls:
    # Send a GET request to the URL to download the text file
    response = requests.get(url)
    # Get the filename from the URL and use it to create the name of the local file
    filename = f"{url.split('/')[-1].split('.')[0]}.txt"
    # Combine the desktop path with the filename to create the full path to the local file
    filepath = os.path.join(desktop_path, filename)
    # Write the contents of the downloaded text file to the local file
    with open(filepath, "w") as f:
        f.write(response.text)

In [ ]:
# create an empty dataframe to hold the data
df2 = pd.DataFrame()

# loop through each text file and read it into a pandas dataframe
for file in glob.glob("//filepathname//*.txt"):
    with open(file, "r") as f:
        content = f.read()
        # create a new dataframe with the file contents and add it to the main dataframe
        df2_new = pd.DataFrame({'text': [content]})
        df2 = pd.concat([df2, df2_new], ignore_index=True)

In [ ]:
# Applying ascii to entire dataframe
df2 = df2.applymap(lambda x: x.encode('ascii', 'ignore').decode('ascii') if isinstance(x, str) else x)

In [ ]:
df2 = df2.rename(columns={'text': 'text2'})

In [ ]:
# Create a new DataFrame with a single row that contains all the text from the original DataFrame
df2 = pd.DataFrame({
    'text2_combined': [df2['text2'].sum()]  # Sum all the strings in the 'text' column and put the result in a list
})

In [ ]:
# Split the text into individual words
df2['words2'] = df2['text2_combined'].str.split()

# Create a new DataFrame with one row for each word
new_df2 = df2.explode('words2')

# Reset the index of the new DataFrame
new_df2 = new_df2.reset_index(drop=True)


In [ ]:
# Define function to convert NLTK part-of-speech tags to WordNet part-of-speech tags
def get_wordnet_pos(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # By default, assume the word is a noun

# Create lemmatizer object
lemmatizer = WordNetLemmatizer()

# Tokenize and lemmatize the 'text' column, and store the result in a new column called 'tokenized_words'
new_df2['tokenized2_words'] = new_df2['words2'].str.lower().apply(nltk.word_tokenize).apply(nltk.pos_tag).apply(lambda x: [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in x])

# SCIENCE FICTION ANALYSIS VAD, TI-IDF

In [ ]:
# Load the VAD scores from the Warriner rescale
vad = pd.read_csv('//filepathname//Warriner_rescale.csv', index_col=0)
vad = vad[['V.Mean.Sum', 'A.Mean.Sum', 'D.Mean.Sum']]
vad.columns = ['valence', 'arousal', 'dominance']

# Define a function to get the VAD scores for a given word
def get_vad_scores(word):
    if word.lower() in vad.index:
        scores = vad.loc[word.lower()].values
        return scores
    else:
        return [np.nan, np.nan, np.nan]
    
# Create a new column in the dataframe to store the VAD scores
new_df2['vad2_scores'] = [[] for _ in range(len(new_df2))]

# Iterate over each row in the dataframe and calculate VAD scores for each tokenized word
for i, row in new_df2.iterrows():
    vad2_scores = []
    for word in row['tokenized2_words']:
        vad2_scores.append(get_vad_scores(word))
    new_df2.at[i, 'vad2_scores'] = vad2_scores

In [ ]:
# Define a function named get_vad_scores that takes a word as input
def get_vad_scores(word):
    # Try to get the VAD scores for the word from the vad DataFrame
    try:
        # Lookup the word in the vad DataFrame and retrieve the VAD scores as a NumPy array
        # The .lower() method is used to convert the word to lowercase for case-insensitive matching
        scores2 = vad.loc[word.lower()].values
        # Return the VAD scores as a NumPy array
        return scores2
    # If the word is not in the vad DataFrame, catch the KeyError exception
    except KeyError:
        # Return None if the word is not in the vad DataFrame
        return None

In [ ]:
#Initialize empty lists to store the VAD scores for each word in the tokenized_words column of new_df

valence_scores2 = []
arousal_scores2 = []
dominance_scores2 = []

#Iterate over each row in new_df

for i, row in new_df2.iterrows():
# Iterate over each word in the tokenized_words column of the current row
    for word in row['tokenized2_words']:
    # Call the get_vad_scores function to get the VAD scores for the current word
        score = get_vad_scores(word)
        # If the get_vad_scores function returns a valid VAD score, append the scores to the appropriate list
        if score is not None:
            valence_scores2.append(score[0])
            arousal_scores2.append(score[1])
            dominance_scores2.append(score[2])

In [ ]:
# Calculate the mean of the valence_scores list
mean_valence2 = np.mean(valence_scores2)

# Calculate the median of the valence_scores list
median_valence2 = np.median(valence_scores2)

# Calculate the standard deviation of the valence_scores list
std_valence2 = np.std(valence_scores2)

# Print the calculated measures
print("Mean Valence Score:", mean_valence2)
print("Median Valence Score:", median_valence2)
print("Standard Deviation of Valence Scores:", std_valence2)

In [ ]:
# Linear regression on valence scores
X = np.array(valence_scores2).reshape(-1, 1)
y = np.array(dominance_scores2)  # replace dominance_scores with your target variable
model = LinearRegression().fit(X, y)
r_sq = model.score(X, y)

print("R-squared value:", r_sq)

In [ ]:
# Linear regression on arousal scores
X = np.array(arousal_scores2).reshape(-1, 1)
y = np.array(dominance_scores2)  # replace dominance_scores with your target variable
model = LinearRegression().fit(X, y)
r_sq = model.score(X, y)

print("R-squared value:", r_sq)

In [ ]:
# Linear regression on arousal scores
X = np.array(valence_scores2).reshape(-1, 1)
y = np.array(arousal_scores2)  # replace dominance_scores with your target variable
model = LinearRegression().fit(X, y)
r_sq = model.score(X, y)

print("R-squared value:", r_sq)

In [ ]:
# create a scatter plot of valence and arousal scores
plt.scatter(valence_scores2, arousal_scores2, color='blue', alpha=0.5)

# set the axis labels and title
plt.xlabel('Valence')
plt.ylabel('Arousal')
plt.title('Science Fiction VAD Scores')

# show the plot
plt.show()

In [ ]:
# create a scatter plot of valence and dominance scores
plt.scatter(valence_scores2, dominance_scores2, color='blue', alpha=0.5)

# set the axis labels and title
plt.xlabel('Valence')
plt.ylabel('Dominance')
plt.title('Science Fiction VAD Scores')

# show the plot
plt.show()

In [ ]:
# create a scatter plot of arousal and dominance scores
plt.scatter(arousal_scores2, dominance_scores2, color='blue', alpha=0.5)

# set the axis labels and title
plt.xlabel('Arousal')
plt.ylabel('Dominance')
plt.title('Science Fiction VAD Scores')

# show the plot
plt.show()

In [ ]:
# Create a new figure for the 3D plot
fig = plt.figure()

# Create a new subplot with 3D projection
ax = fig.add_subplot(111, projection='3d')

# Plot the VAD scores for each word as a scatter plot
ax.scatter(valence_scores2, arousal_scores2, dominance_scores2)

# Set the labels for the x, y, and z axes
ax.set_xlabel('Valence')
ax.set_ylabel('Arousal')
ax.set_zlabel('Dominance')

# Display the plot
plt.show()

In [ ]:
# Create a trace for the scatter plot
trace = go.Scatter3d(
    x=valence_scores2,
    y=arousal_scores2,
    z=dominance_scores2,
    mode='markers',
    marker=dict(
        size=5,
        color=valence_scores2,  # Set color to valence scores
        colorscale='reds',  # Set the color scale to red-pink
        opacity=0.8
    )
)

# Create the layout for the plot
layout = go.Layout(
    scene=dict(
        xaxis=dict(title='Valence'),
        yaxis=dict(title='Arousal'),
        zaxis=dict(title='Dominance')
    ),
    margin=dict(l=0, r=0, b=0, t=0)
)

# Create the figure and add the trace and layout
fig = go.Figure(data=[trace], layout=layout)

# Display the plot
fig.show()


In [ ]:
# get a random sample of 10 documents
documents2 = new_df2['text2_combined'].sample(n=10).tolist()

# create a list of stop words
stop_words2 = set(stopwords.words('english'))

# tokenize each document and remove stop words
texts2 = [[word for word in word_tokenize(document.lower()) if word not in stop_words2] for document in documents2]

# create a dictionary from the tokenized texts
dictionary2 = corpora.Dictionary(texts2)

# create a corpus from the dictionary and tokenized texts
corpus = [dictionary2.doc2bow(text) for text in texts2]

# build the LDA model with 5 topics
num_topics = 5
lda_model = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary2, num_topics=num_topics, passes=10, random_state=42)

# print the top 10 words for each topic
for i, topic in lda_model.show_topics(num_topics=num_topics, num_words=10, formatted=False):
    print(f'Topic {i}: {" ".join([word[0] for word in topic])}')

In [ ]:
# create the matrix
doc_topic_matrix = []
for i, doc in enumerate(corpus):
    topic_probs = lda_model.get_document_topics(doc, minimum_probability=0)
    topic_probs_dict = dict(topic_probs)
    row = [topic_probs_dict.get(j, 0) for j in range(num_topics)]
    doc_topic_matrix.append(row)

# create the heatmap
sns.heatmap(doc_topic_matrix, cmap='YlGnBu')
plt.xlabel('Topic2')
plt.ylabel('Document2')
plt.title(f'Science Fiction Topic Probabilities for {len(documents2)} Documents')
plt.show()

In [ ]:
new_df.to_csv('QT2005_Assessment#3_21010680514_Fiction.csv', index=False)

In [ ]:
new_df2.to_csv('QT2005_Assessment#3_21010680514_Science_Fiction.csv', index=False)